In [9]:
import pandas as pd
import scipy.stats as sps
import numpy as np
d1 = {'one': pd.Series(range(6), index=list('abcdef')),
     'two': pd.Series(range(7), index=list('abcdefg')),
     'three': pd.Series(sps.norm.rvs(size=7), index=list('abcdefg'))}
df1 = pd.DataFrame(d1).reset_index()


d2 = {'second_index': pd.Series(['a', 'b', 'd', 'c', 'd', 'c', 'k', 'l']),
     'four': pd.Series(range(10, 18)),
     'five': pd.Series(sps.norm.rvs(size=8))}
df2 = pd.DataFrame(d2)
df2

,second_index,four,five
0,a,10,0.883922
1,b,11,1.143409
2,d,12,1.319853
3,c,13,-1.633404
4,d,14,-0.352832
5,c,15,-2.412491
6,k,16,0.380302
7,l,17,-0.859375


### Пример формата запроса:

SELECT tab1.column1 as c1, tab2.column2 as c2, tab1.column1+tab2.column2 as sum12
FROM tab1 RIGHT JOIN tab2 ON tab1.ind1=tab2.ind2
WHERE tab2.four>5 ORDER BY tab2.column3 ASC, tab1.column4 DESC

Реализовать какой-то больший функционал, чем указан в примере запроса не нужно. Общий функционал: новые имена столбцов, арифметические операции на столбцах, объединение таблиц (не только правое!), фильтр по условию, сортировка.

Чтобы легче было парсить, считаем, что пользователь всегда пишет запрос красиво и корректно (как в примере).

In [10]:
tab_dict = {'tab1': df1, 'tab2': df2}

In [11]:
def SQL_query(query, tab_dict):
    operations = {'FROM':'', 'JOIN' : '','ON' : '', 'WHERE' : '', 'ORDER' : '', }
    query_lst = np.array(query.split())
    keys_arr = np.array(list(operations.keys()))
    values_arr = np.array(list(operations.values()))
    for i in list(operations.keys()):
            operations[i] = np.argmax((np.char.find(query_lst, i))) \
            if np.argmax((np.char.find(query_lst, i))) != 0 else None
    def join(join_lst, tab_dict, keys : str):
        keys = keys[0].split('=')
        for i in range(len(keys)):
            keys[i] = keys[i].split('.')
            keys[i] = keys[i][1]
        df1_ind = np.argmax(np.char.find(join_lst, list(tab_dict.keys())[0]))
        df2_ind = np.argmax(np.char.find(join_lst, list(tab_dict.keys())[1]))
        if df1_ind < df2_ind: 
            type_of_join = join_lst[(df1_ind + 1) : (df2_ind - 1)]
            if len(type_of_join) == 0:
                return tab_dict[join_lst[df1_ind]].merge(right = tab_dict[join_lst[df2_ind]],\
                                                        left_on = keys[0], right_on = keys[1])
            else:
                return tab_dict[join_lst[df1_ind]].merge(right = tab_dict[join_lst[df2_ind]],how = type_of_join[0].lower(),\
                                                        left_on = keys[0], right_on = keys[1])
        else:
            type_of_join = join_lst[(df2_ind + 1) : (df1_ind - 1)]
            if len(type_of_join) == 0:
                return tab_dict[join_lst[df2_ind]].merge(right = tab_dict[join_lst[df1_ind]],\
                                                        left_on = keys[0], right_on = keys[1])
            else:
                return tab_dict[join_lst[df1_ind]].merge(right = tab_dict[join_lst[df2_ind]],how = type_of_join[0].lower(),\
                                                        left_on = keys[0], right_on = keys[1])
            
    
    
    def where(dataframe, condition: str):
        operators = {'<' : lambda x: dataframe[x.split('<')[0]] < float(x.split('<')[1]),
                    '>' : lambda x: dataframe[x.split('>')[0]] > float(x.split('>')[1]),
                    '=': lambda x: dataframe[x.split('=')[0]] == float(x.split('=')[1]),
                    '>=': lambda x: dataframe[x.split('>=')[0]] >= float(x.split('>=')[1]),
                    '<=': lambda x: dataframe[x.split('<=')[0]] >= float(x.split('<=')[1])}

        condition = condition.split('.')[1]
        for operator in list(operators.keys())[::-1]:
            if operator in condition:
                bool_list_condition = list(operators[operator](condition))
                return dataframe.loc[bool_list_condition, :]        
    
    
    def order_by(lst_to_sort : list, dataframe):
        lst_to_sort = (' '.join(lst_to_sort)).split(',')
        asc_lst = []
        for i in lst_to_sort:
            if 'DESC' in i:
                asc_lst.append(False)
            else:
                asc_lst.append(True)
        lst_to_sort = ''.join(lst_to_sort).replace('DESC', '').replace('ASC', '').replace(list(tab_dict.keys())[0] + '.', '')\
        .replace(list(tab_dict.keys())[1] + '.', '').split()
        return dataframe.sort_values(by = lst_to_sort, ascending = asc_lst )
    
    def select(lst_to_sel, df):
        operators = {'+': lambda x: df[x.split('+')[0]] + df[x.split('+')[1]] , '-' : lambda x:df[x.split('+')[0]] - df[x.split('+')[1]]}
        itog_lst = []
        
        for col_str in lst_to_sel:
            if '+' not in col_str and '-' not in col_str:
                if 'as' not in col_str:
                    itog_lst.append(col_str)
                else:
                    col_str = col_str.split(' as ')
                    df = df.rename(columns = {col_str[0] : col_str[1]})
                    itog_lst.append(col_str[1])
            elif '+' in col_str or '-' in col_str:
                if 'as' in col_str:
                    col_str = col_str.split(' as ')
                    
                    for char in col_str[0]:
                        if char in list(operators.keys()):
                            df[col_str[1]] = operators[char](col_str[0])
                            itog_lst.append(col_str[1])
                else:
                    for char in col_str:
                        if char in list(operators.keys()):
                            df['noname'] = operators[char](col_str)
                            itog_lst.append('noname')
        return df.loc[:, itog_lst]
    if operations['JOIN'] != None:
        first_not_none = keys_arr[3:][values_arr[3:] != None] #возвращает список ключей('WHERE' и 'ORDER'), словаря operations, у которых не None
        try:
            first_not_none = first_not_none[0]
            first_ind = operations[first_not_none]
            keys = query_lst[(operations['ON'] + 1) : first_ind]
            df = join(query_lst[(operations['FROM'] + 1):operations['ON']], tab_dict, keys)
        except IndexError:
            keys = query_lst[operations['ON']:]
            df = join(query_lst[(operations['FROM'] + 1):operations['ON']], tab_dict, keys)
            
    else:
        df = tab_dict[query_lst[(operations['FROM'] + 1)]]
    
    
    if operations['WHERE'] != None:
        condition_index = operations['WHERE'] + 1
        condition = query_lst[condition_index]
        df = where(df, condition)
    
    if operations['ORDER'] != None:
        lst_to_sort = query_lst[(operations['ORDER'] + 2):] #2 так как после "ORDER" еще идет "BY"
        df = order_by(lst_to_sort, df)
    
    lst_to_select = query_lst[1: operations['FROM']]
    lst_to_select = ' '.join(lst_to_select).replace(list(tab_dict.keys())[0] + '.', '')\
    .replace(list(tab_dict.keys())[1] + '.', '').split(', ')
    
    return select(lst_to_select, df)

    
    
        
         

In [12]:
query = 'SELECT tab1.one as c1, tab2.five as c2, tab1.three+tab2.four as sum12 FROM tab1 JOIN tab2 ON tab1.index=tab2.second_index WHERE tab2.one>=2 ORDER BY tab2.five ASC'

In [13]:
SQL_query(query, tab_dict)

,c1,c2,sum12
3,2.0,-2.412491,17.191610
2,2.0,-1.633404,15.191610
5,3.0,-0.352832,14.721438
4,3.0,1.319853,12.721438
